# Bayesian Linear Modeling

Bayesian approaches to estimation offer a great deal of flexibility in modeling a given data generating process. However, few introductions to econometrics offer a bayesian perspective at the outset. This creates an unfortunate gap in understanding the ways in which bayesian and frequentist approaches are related. Relatedly, there is often another gap in understanding how bayesian approaches are actually operationalized. 

The purpose of this notebook is to demonstrate that bayesian approaches to linear regression can service the same needs as canonical frequentist approaches. They also buy us ways of directly modeling uncertainty. To show this, we'll use the `mpg` dataset (provided via `seaborn`) to explore the relationship between miles per gallon and vehicle weight. Specifically, we will estimate that relationship using Ordinary Least Squares regression in three ways:

1. OLS analytically derived from the [normal equation](https://mathworld.wolfram.com/NormalEquation.html)
2. OLS estimated by maximum likelihood
3. OLS estimated via posterior sampling in bayesian linear regression

In [3]:
import seaborn as sb
from matplotlib.figure import Figure
from matplotlib.axes import Axes
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
from typing import List, Tuple
import pandas as pd
import xarray as xr
from statsconcepts.regression import OLS, RegModel
from dataclasses import dataclass

%pylab inline

colors: List[str] = sb.color_palette("Set1")

ImportError: cannot import name 'naming' from 'tensorflow.python.autograph.core' (/usr/local/Caskroom/miniconda/base/envs/tf/lib/python3.7/site-packages/tensorflow/python/autograph/core/__init__.py)

# Input Data

The `mpg` dataset can be loaded directly from the `seaborn` package. Note that our goal is to be able to say things like when the weight of a car is $x$ pounds above average, we tend to see that miles per gallon is $y$ miles below average. To facilitate this framing, we are going to demean our variables of interest.

In [ ]:
ds: xr.Dataset = xr.Dataset(sb.load_dataset("mpg"))
ds["demeaned_mpg"] = ds["mpg"] - ds["mpg"].mean()
ds["demeaned_weight"] = ds["weight"] - ds["weight"].mean()

ds.head()

As can be seen in the plot below, lower weights are associated with higher miles per gallon and vice versa.

In [ ]:
def plot_data(ds: xr.Dataset, x: str, y: str, figsize: Tuple[int, int] = (15, 10), **kwargs) -> Axes:
    fig: Figure
    ax: Axes
    fig, ax = plt.subplots(figsize=figsize)
    sb.scatterplot(x=ds[x], y=ds[y], ax=ax, color="white", ec="red")
    return ax
    
plot_data(ds, "demeaned_weight", "demeaned_mpg")

# Analytic Solution

The first approach we typically learn is to directly use the regression equation $y = X'\beta + \epsilon$ to derive the value of the weight vector $\beta$  in terms of the observed data. 

$$
\begin{align}
    y &= X' \beta  + \epsilon \\
    \epsilon &= y - X' \beta \\
    \epsilon'\epsilon &= y'y - 2X'y \beta + \beta'X'X \beta \\
    \frac{\partial}{\partial \beta} \epsilon'\epsilon &= \frac{\partial}{\partial \beta}y'y - 2X'y \beta + \beta'X'X \beta \\
    0 &= -2X'y + 2X'X \beta \\
    2X'X \beta &= 2X'y \\ 
    \beta &= (X'X)^{-1}X'y
\end{align}
$$

The last line is the OLS estimator of the extent to which the regressors $X$ covary with the dependent data $y$ on average. We can use this definition to directly to compute our weight vector. In `fit_exec()` below, the variable `cov` (for covariance) captures the numerator ($X'Y$) and the variable `var_design` (for variance of the design matrix) captures the denominator ($X'X$).

```python
@dataclass
class OLS:

    @staticmethod
    def fit(formula: str, data: xr.Dataset) -> RegModel:
        y: DesignMatrix
        x: DesignMatrix
        y, x = dmatrices(formula, data)
        y_tensor: tf.Tensor = tf.constant(y)
        x_tensor: tf.Tensor = tf.constant(x)
        betas: tf.Tensor = OLS.fit_exec(y_tensor, x_tensor)
        out: RegModel = RegModel(data, y, x, betas, "Direct Ordinary Least Squares")
        return out

    @staticmethod
    def fit_exec(y: tf.Tensor, x: tf.Tensor) -> tf.Tensor:
        cov: tf.Tensor = tf.linalg.matmul(tf.transpose(x), y)
        var_design: tf.Tensor = tf.linalg.matmul(tf.transpose(x), x)
        inv_var_design: tf.Tensor = tf.linalg.inv(var_design)
        betas: tf.Tensor = tf.matmul(inv_var_design, cov)
        return betas
```



In [ ]:
analytic: RegModel = OLS.fit("demeaned_mpg ~ demeaned_weight", ds)
    
analytic

As can be seen, our intercept is effectively zero (appropriate since we demeaned the data) and the slope is about -0.008. In other words, on average, cars that are one pound above average weight are associated with being able to get -0.008 fewer miles to the gallon. Equivalently, cars that are about 130 pounds above average weight tend to get one mile less per gallon.

In [ ]:
def plot_est(betas: tf.Tensor, support: tf.Tensor, ax: Axes) -> Axes:
    map_func: Callable[[float], float] = lambda x: betas[0] + betas[1] * x
    ys: tf.Tensor = tf.map_fn(fn=map_func, elems=support)
    ax.plot(support, ys, linestyle="--", color="grey")
    return ax
    
plot_est(
    betas=analytic.betas,
    support=tf.linspace(ds["demeaned_weight"].min().data, ds["demeaned_weight"].max().data, 100),
    ax=plot_data(ds, "demeaned_weight", "demeaned_mpg")
)

Given our estimate of the weight vector $\beta$ we can compute the variance in the errors (the gap between our estimate and observed data). This variance is our estimate of the variance in $\epsilon$.

In [ ]:
def compute_std_dev(beta: tf.Tensor, y: tf.Tensor, x: tf.Tensor) -> tf.Tensor:
    # Apply the weight parameters to the data
    apply_weights: Callable[[float], float] = lambda datum: beta[0] + beta[1] * datum
    weighted_data: tf.Tensor = tf.map_fn(apply_weights, x)
        
    # Calculate variance in errors (note the avg is zero)
    errors: tf.Tensor = y - weighted_data
    variance: tf.Tensor = tf.reduce_mean (
        tf.pow(errors, 2)
    )
    return tf.pow(variance, 0.5)
        
compute_std_dev(
    beta=analytic.betas,
    y=tf.constant(ds["demeaned_mpg"].data),
    x=tf.constant(ds["demeaned_weight"].data)
)

As can be seen, the variance in $\epsilon$ is roughly 10.15.

# Maximum Likelihood Solution

When estimating by maximum likelihood, our approach shifts a bit. We don't focus on analytically solving for $\beta$, but rather ask ourselves the following:

> Of all the values that $\beta$ could take, which value is most consistent with the data we observe?

To answer this question, we need to know how plausible the data are given some value for $\beta$. This concept is known as statistical likelihood. Once we can evaluate the likelihood for some value of $\beta$, we can do for all (relevant) values of $\beta$. After doing so, our estimate is just the most plausible value (i.e. the maximum likelihood estimate).

## How do we determine the likelihood at a given value of $\beta$?

Suppose we had some random variable that was distributed normally with zero mean and a standard deviation of one (i.e. $x \sim N(0, 1)$).

In [ ]:
std_norm: tfd.Normal = tfd.Normal(loc=0., scale=1.)
    
def plot_univ(xs: tf.Tensor, figsize: Tuple[int, int] = (15, 10), **kwargs) -> Axes:
    fig: Figure
    ax: Axes
    fig, ax = plt.subplots(figsize=figsize)
    sb.kdeplot(xs, ax=ax)
    return ax
    
    
plot_univ(std_norm.sample(10000))

Given that we know the parameters that define the distribution of the random variable (`loc` is $\mu$ and `scale` is $\sigma$), we can ask the following:

> Given that this random variable is distributed normally with with our values for $\mu$ and $\sigma$, if we were to take a sample, what is the likelihood that the sampled value would be 0? 1? -2?

To find this out, we need only evaluate the probability density function for our distribution at those values. The value yielded is a relative measure of sampling plausibility (i.e. the likelihood). For consistency and computational reasons that will later become relevant, we'll use the logged likelihood value instead of the untransformed likelihood. Since logging is a [monotonic transformation](https://en.wikipedia.org/wiki/Monotonic_function) the ordering is preserved, so we still have the relative relationships we are seeking.

In [ ]:
def plot_ll_values(dist: tfd.Distribution, values: tf.Tensor, ax: Axes, lab_y: float = 0.35, lab_dx: float = 0.1) -> Axes:
    with_log_lik: Tuple[float, float] = [(orig.numpy(), dist.log_prob(orig).numpy()) for orig in values]
    for orig, ll in with_log_lik:
        ax.axvline(x=orig, color="orange")
        ax.annotate("{:.3f}".format(ll), xy=(orig - lab_dx, lab_y), rotation=90)
    return ax

plot_ll_values(
    dist=std_norm,
    values=tf.constant([0., 1., -2.]),
    ax=plot_univ(std_norm.sample(10000))
)

As can be seen, and should accord with intuition, 0 is the value that is most plausibly drawn from the distribution. It has the least negative log likelihood. Drawing 1 is less likely, and drawing -2 is less likely still.

So what does this have to do with $\beta$? Recall our regression equation:

$$y = X'\beta + \epsilon$$

One of the fundamental assumptions that underlies OLS is that our error term, $\epsilon$ is normally distributed with mean zero and some positive variance (i.e. $\epsilon \sim N(0, \sigma)$). If $\epsilon$ does not add anything on average, then the average value of $y$ must come from $X'\beta$. Moreover, $X'\beta$ is a computed value, not a random variable. All of the variation on the right side is coming from that $\sigma$ in the distribution of $\epsilon$. Consequently, a different way to express the regression equation is as follows:

$$y \sim N(X'\beta, \sigma)$$

Written in this form, a couple things become clear:

1. We now have distributions that we can evaluate at several points (specifically those coming from the data $X$).
2. We can set values for the parameters $\beta$ and $\sigma$ to *define* specific instances of those distributions.

We are doing the same thing we did with our standard normal example, but now we are using our parameters to calculate `loc` and directly define `scale`.

In [ ]:
def ols_ll(beta: tf.Tensor, sigma: float, y: tf.Tensor, x: tf.Tensor) -> tf.Tensor:
    # Apply the weight parameters to the data (testing the plausibility of these)
    apply_weights: Callable[[float], float] = lambda datum: beta[0] + beta[1] * datum
    weighted_data: tf.Tensor = tf.map_fn(apply_weights, x)
        
    # Define the distribution using the expected value of the observed response data and sigma
    avg_data: float = tf.reduce_mean(y)
    dist: tfd.Normal = tfd.Normal(avg_data, sigma)
        
    # Compute the log likelihood
    data_lls: tf.Tensor = tf.map_fn(dist.log_prob, weighted_data)
    total_ll: tf.Tensor = tf.reduce_sum(data_lls)
    return total_ll

@dataclass
class SimpleParams:
    beta0: float
    beta1: float
    sigma: float
        
params: List[SimpleParams] = [
    SimpleParams(0., -0.008, 10.15),
    SimpleParams(0., -0.008, 7.8),
    SimpleParams(0., -0.008, 100.15),
    SimpleParams(100., -0.008, 10.15),
    SimpleParams(0., 1, 10.15),
]

for p in params:
    print(p)
    print(ols_ll(
        beta=tf.constant([p.beta0, p.beta1], dtype=tf.float64),
        sigma=p.sigma,
        y=tf.constant(ds["demeaned_mpg"].data),
        x=tf.constant(ds["demeaned_weight"].data)
    ))
    print("\n")

As can be seen choosing parameter values close to what we estimated analytically yields the least negative number. Clearly these are just spot checks, but really it's foreshadowing the eventual result.

## How do we determine the maximum likelihood estimate of $\beta$?

One way to determine the MLE is to create a mesh. In this case, it would be three dimensions. Along one axis of the cube, we'd have a bunch of values for the intercept $\beta_0$. Along another we'd have a bunch of values for $\beta_1$, and along the last a bunch of values for $\sigma$. The intersections of all the values in the cube could then be used to calculate the likelihoods given the data. The intersection with the highest likelihood wins and we take the values of the parameters at that intersection as our MLE.

The thing is, that can get pretty computationally intensive depending on how many parameters we have and how fine grained we want the mesh to be. A more efficient solution would be to represent variation in our parameters as a likelihood function. If we can do that, we can differentiate that function and use the output to determine the maximum value. This turns out to be much more efficient and one of the great selling points for Tensorflow is its automatic differentiation capability. In other words, we can get Tensorflow to the heavy lifting on the calculus front.

All that said, let's

In [ ]:
y: tf.Tensor = tf.constant(ds["demeaned_mpg"].data)
x: tf.Tensor = tf.concat([
    tf.ones((len(y), 1), dtype=tf.float64), 
    tf.reshape(ds["demeaned_weight"].data, shape=(len(y), 1))
], axis=1)

In [ ]:
@dataclass
class OLSmle:
    y_arr: xr.DataArray
    x_arr: xr.DataArray
    
    def __post_init__(self) -> None:
        self.y: tf.Tensor = tf.constant(self.y_arr.data)
        self.x: tf.Tensor = tf.concat([
            tf.ones((len(y), 1), dtype=tf.float64), 
            tf.reshape(self.x_arr.data, shape=(len(y), 1))
        ], axis=1)
    
    def neg_ols_ll(self, params: tf.Tensor) -> tf.Tensor:
        """Requires params vector to end with sigma"""
        beta_len: int = params.shape[0] - 1
        beta: tf.Tensor = tf.cast(tf.reshape(params[:-1], (beta_len, 1)), dtype=tf.float64)
        sigma: tf.Tensor = tf.cast(params[-1], dtype=tf.float64)
        mu: tf.Tensor = tf.matmul(self.x, beta)
#         print(beta_len)
#         print(beta)
#         print(sigma)
#         print(mu)
        return -tf.reduce_sum(tfd.Normal(tf.reduce_mean(self.y), sigma).log_prob(mu))

    def neg_ols_ll_surface(
        self,
        beta0_vals: tf.Tensor,
        beta1_vals: tf.Tensor,
        sigma_vals: tf.Tensor
    ) -> tf.Tensor:
        """
        There must be a better way to do this, but not sure how to get
        around the need for conformant matrix multiplication (beta against data)
        """
        
        print(beta0_vals)
        print(beta1_vals)
        print(sigma_vals)
        out: tf.Tensor = tf.zeros(shape=(
            beta0_vals.shape[0],
            beta1_vals.shape[0],
            sigma_vals.shape[0]
        ))
        
        for i, b0 in enumerate(beta0_vals):
            for j, b1 in enumerate(beta1_vals):
                for k, s in enumerate(sigma_vals):
                    print(b0, b1, s)
                    ps: tf.Tensor = tf.constant([b0.numpy(), b1.numpy(), s.numpy()])
                    print(ps)
                    out[i, j, k] = self.neg_ols_ll(params=ps) #breaks on index assignment
        
        return out
    
    def neg_ols_ll_w_gradient(self, params: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
        """
        Tensorflow will automatically compute the partial derivatives
        associated with out log-likelihood function we have deployed. We
        can get the gradients for each parameter returned to us when
        we compute the negative log likelihood.
        """
        return tfp.math.value_and_gradient(self.neg_ols_ll, params)
    
    def fit(
        self, 
        init_param_vals: tf.Tensor = tf.constant([0., 0., 1.], dtype=tf.float64),
        tol: float = 1e-8,
        **kwargs
    ) -> tf.Tensor:
        opti_results: tf.Tensor = tfp.optimizer.bfgs_minimize(
            value_and_gradients_function=self.neg_ols_ll_w_gradient,
            initial_position=init_param_vals,
            tolerance=tol,
            **kwargs
        )
        status: str = f"""
        RESULTS
        Converged: {opti_results.converged.numpy()}
        Location of Minimum: {opti_results.position.numpy()}
        Number of Iterations: {opti_results.num_iterations.numpy()}
        """
        print(status)
        return opti_results.position.numpy()
        
    
mle: OLSmle = OLSmle(
    y_arr=ds["demeaned_mpg"],
    x_arr=ds["demeaned_weight"]
)
    
mle.neg_ols_ll_w_gradient(params=tf.constant([0., -0.8, 10.15], dtype=tf.float64))

In [ ]:
for i in range(-10, 10):
    print(i, mle.neg_ols_ll(params=tf.constant([float(i), -0.008, 10.15], dtype=tf.float64)))

In [ ]:
for i in range(-10, 10):
    print(i, mle.neg_ols_ll(params=tf.constant([0., float(i), 10.15], dtype=tf.float64)))

In [ ]:
for i in range(-10, 10):
    print(i, mle.neg_ols_ll(params=tf.constant([0., -0.008, float(i)], dtype=tf.float64)))

In [ ]:
mle.neg_ols_ll_w_gradient(mle.fit(init_param_vals=tf.constant([1., -1., 8.15], dtype=tf.float64), tol=1e-15))

# Bayesian Solution

The bayesian approach to estimation overlaps with the MLE approach to the extent that there is still heavy reliance on the likelihood function. The difference now will be that the likelihood serves the purpose of allowing data to update our prior beliefs about our parameter values. Instead of making no claims about prior distributions (i.e. implicitly assuming all values are equally plausible), we will directly model our paramsters as random variables.

In [ ]:
def bayes_likelihood(beta0: tf.Tensor, beta1: tf.Tensor, sigma: tf.Tensor) -> tfd.Independent:
    beta = tf.concat([
        tf.reshape(beta0, shape=(1, 1)),
        tf.reshape(beta1, shape=(1, 1))
    ], axis=0)
    out: tfd.Independent = tfd.Independent(tfd.Normal(
        loc=tf.matmul(x, beta),
        scale=sigma
    ), reinterpreted_batch_ndims=1)
    return out
    

model_ols: tfd.JointDistributionNamed = tfd.JointDistributionNamed(dict(
    beta0 = tfd.Normal(loc=tf.cast(0., dtype=tf.float64), scale=1.),
    beta1 = tfd.Normal(loc=tf.cast(0., dtype=tf.float64), scale=0.01),
    sigma = tfd.Uniform(low=tf.cast(7., dtype=tf.float64), high=8.),
    likelihood = bayes_likelihood
))
    
model_ols.parameters

This `JointDistribution` formulation allows us to directly sample from our composite prior.

In [ ]:
model_ols.sample()

In [ ]:
tf.reduce_sum(model_ols.log_prob({
    "sigma": 7.,
    "beta0": 0.,
    "beta1": -0.008,
    "y": y
}))

In [ ]:
def target_log_prob_fn(beta0: float, beta1: float, sigma: float) -> tf.Tensor:
    params: Dict[str, float] = dict(
        beta0=beta0,
        beta1=beta1,
        sigma=sigma,
        y=y
    )
    print({k: v for k, v in params.items() if k != "y"})
    out: tf.Tensor = tf.reduce_sum(model_ols.log_prob(params))
    return out

# target_log_prob_fn(0., -0.008, 7.86)
num_results: int = int(1e2)
num_burn_in_steps: int = int(1e1)

hmc_kernel: tfp.mcmc.HamiltonianMonteCarlo = tfp.mcmc.HamiltonianMonteCarlo(
    target_log_prob_fn=target_log_prob_fn,
    step_size=1.0,
    num_leapfrog_steps=3
)
    
adaptive_hmc_kernel: tfp.mcmc.SimpleStepSizeAdaptation = tfp.mcmc.SimpleStepSizeAdaptation(
    inner_kernel=hmc_kernel,
    num_adaptation_steps=int(num_burn_in_steps * 0.8)
)
    
mh_kernel: tfp.mcmc.MetropolisHastings = tfp.mcmc.MetropolisHastings(
    tfp.mcmc.UncalibratedHamiltonianMonteCarlo(
        target_log_prob_fn=target_log_prob_fn,
        step_size=0.1,
        num_leapfrog_steps=3
    )
)
    
@tf.function
def run_chain():
    
    samples, is_accepted = tfp.mcmc.sample_chain(
        num_results = num_results,
        current_state=[
            tf.constant([0.], dtype=tf.float64),
            tf.constant([1.], dtype=tf.float64),
            tf.constant([7.], dtype=tf.float64),
        ],
        num_burnin_steps=num_burn_in_steps,
        kernel=adaptive_hmc_kernel,
        trace_fn=lambda _, pkr: pkr.inner_results.is_accepted
    )
    return samples

In [ ]:
run_chain()

In [ ]:
import pandas as pd
